In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

22/02/21 14:13:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-21 14:14:57--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.136.209
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.136.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv’

fhvhv_tripdata_2021 100%[===================>] 717.48M  30.1MB/s    in 24s     

2022-02-21 14:15:22 (30.0 MB/s) - ‘fhvhv_tripdata_2021-01.csv’ saved [752335705/752335705]



In [5]:
!wc -l fhvhv_tripdata_2021-01.csv

11908469 fhvhv_tripdata_2021-01.csv


In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [7]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [9]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd

In [12]:
df_pandas = pd.read_csv('head.csv')

In [13]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [16]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [20]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [23]:
df = df.repartition(24)

In [24]:
df.write.parquet('fhvhv/2021/01')

In [25]:
df = spark.read.parquet('fhvhv/2021/01/')

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [30]:
from pyspark.sql import functions as F

In [32]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-07|  2021-01-07|         142|         230|
| 2021-01-01|  2021-01-01|         133|          91|
| 2021-01-01|  2021-01-01|         147|         159|
| 2021-01-06|  2021-01-06|          79|         164|
| 2021-01-04|  2021-01-04|         174|          18|
| 2021-01-04|  2021-01-04|         201|         180|
| 2021-01-04|  2021-01-04|         230|         142|
| 2021-01-03|  2021-01-03|         132|          72|
| 2021-01-01|  2021-01-01|         188|          61|
| 2021-01-04|  2021-01-04|          97|         189|
| 2021-01-01|  2021-01-01|         174|         235|
| 2021-01-05|  2021-01-05|          35|          76|
| 2021-01-06|  2021-01-06|          35|          39|
| 2021-01-04|  2021-01-04|         231|          13|
| 2021-01-02|  2021-01-02|          87|         127|
| 2021-01-02|  2021-01-02|          17|       

In [33]:
def silly_conv(base_num):
    num = int(base_num[1:])
    if num %7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [34]:
silly_conv('B02800')

's/af0'

In [35]:
silly_conv_udf = F.udf(silly_conv, returnType=types.StringType())

In [36]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', silly_conv_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-07|  2021-01-07|         142|         230|
|  e/9ce| 2021-01-01|  2021-01-01|         133|          91|
|  e/acc| 2021-01-01|  2021-01-01|         147|         159|
|  e/b35| 2021-01-06|  2021-01-06|          79|         164|
|  s/b44| 2021-01-04|  2021-01-04|         174|          18|
|  e/b3b| 2021-01-04|  2021-01-04|         201|         180|
|  e/9ce| 2021-01-04|  2021-01-04|         230|         142|
|  e/b38| 2021-01-03|  2021-01-03|         132|          72|
|  s/af0| 2021-01-01|  2021-01-01|         188|          61|
|  e/9ce| 2021-01-04|  2021-01-04|          97|         189|
|  e/acc| 2021-01-01|  2021-01-01|         174|         235|
|  a/b37| 2021-01-05|  2021-01-05|          35|          76|
|  e/9ce| 2021-01-06|  2021-01-06|          35|          39|
|  e/b42| 2021-01-04|  2